In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import glob
import pandas as pd
from common_functions import get_parties, get_election_results, process_election

In [3]:
# These files come from http://www.infoelectoral.mir.es/infoelectoral/min/areaDescarga.html.
# Scroll down to "Extracción de datos" to obtain the zip files with the info at the
# ballot box level.
datafiles = glob.glob("/home/chema/Dropbox/data/elecciones/*.zip")
print(datafiles)

['/home/chema/Dropbox/data/elecciones/04201105_MESA.zip', '/home/chema/Dropbox/data/elecciones/04201505_MESA.zip', '/home/chema/Dropbox/data/elecciones/02201512_MESA.zip', '/home/chema/Dropbox/data/elecciones/02201606_MESA.zip']


De acuerdo a [este archivo](http://www.ine.es/daco/daco42/codmun/codmun11/11codmunmapa.htm), Madrid es la provincia 28, municipio 079. Tenemos que filtrar usando estos códigos.

In [86]:
# Find the doomed ballot boxes in the 2011 elections
el2011 = process_election(datafiles[0])

In [87]:
# Need to get the percentage of vote for a given table + section
el2011_total = el2011.groupby(['section_code', 'table_code', 'dist_code'])\
    .agg({'votes': 'sum'})\
    .rename(columns = {'votes': 'total_votes'})\
    .reset_index()
#el2011_total['total_votes'] = el2011_total['votes']
#del el2011_total['votes']
el2011 = pd.merge(el2011, el2011_total)
el2011['pct_votes'] = el2011['votes'] / el2011['total_votes']

In [35]:
# Now find the boxes that had the most vote to Falange
el2011.loc[el2011['party_name'] == "LA FALANGE", :].sort_values('pct_votes', ascending = False).head(5)

,dist_code,party_code,party_name,prov_code,section_code,table_code,town_code,votes,total_votes,pct_votes
27283,08,122620,LA FALANGE,28,012,U,079,349,702,0.497151
43033,11,122620,LA FALANGE,28,007,B,079,184,396,0.464646
57333,13,122620,LA FALANGE,28,203,A,079,84,404,0.207921
53683,13,122620,LA FALANGE,28,061,U,079,5,301,0.016611
1733,01,122620,LA FALANGE,28,062,U,079,8,487,0.016427


In [85]:
dist_code = '03'
section_code = '078'
table_code = 'B'
summary = el2011.loc[(el2011['section_code'] == section_code) \
                      & (el2011['dist_code'] == dist_code) \
                      & (el2011['table_code'] == table_code), :]\
    .sort_values('pct_votes', ascending = False)
display(summary)
summary.to_csv(f'/tmp/resultados_distrito_{dist_code}_seccion_{section_code}_mesa_{table_code}.csv')
# 1.0
# print(el2011.loc[(el2011['section_code'] == '012') & (el2011['dist_code'] == '08'), :].pct_votes.sum())

,dist_code,party_code,party_name,prov_code,section_code,table_code,town_code,votes,total_votes,pct_votes,threshold
57303,03,123076,PARTIDO POPULAR,28,078,B,079,347,477,0.727463,10.072813
50609,03,123058,PARTIDO OBRERO SOCIALISTA INTERNACIONALISTA,28,078,B,079,51,477,0.106918,0.014158
77385,03,123780,UNION PROGRESO Y DEMOCRACIA,28,078,B,079,41,477,0.085954,1.586954
23833,03,122515,IZQUIERDA UNIDA-LOS VERDES,28,078,B,079,20,477,0.041929,2.258672
404,03,120694,ALTERNATIVA ESPAÑOLA,28,078,B,079,3,477,0.006289,0.062319
53956,03,123068,PARTIDO PIRATA,28,078,B,079,3,477,0.006289,0.063297
80732,03,123960,FAMILIA Y VIDA,28,078,B,079,3,477,0.006289,0.031368
10445,03,121446,ECOLO VERDES,28,078,B,079,3,477,0.006289,0.183961
3751,03,121182,CIUDADANOS EN BLANCO,28,078,B,079,2,477,0.004193,0.144577
13792,03,121696,FALANGE AUTENTICA,28,078,B,079,1,477,0.002096,0.012723


In [91]:
# Sanity check: do we get the same global numbers as the webpage reports
# http://www.infoelectoral.mir.es/infoelectoral/min/busquedaAvanzadaAction.html
el2011_stats = el2011.groupby('party_name')\
    .agg({'pct_votes': 'mean'})\
    .rename(columns = {'pct_votes': 'mean'})\
    .reset_index()
el2011_stats['threshold'] = 20 * el2011_stats['mean']
el2011 = pd.merge(el2011, el2011_stats)
# Yes, we do.

In [92]:
el2011.loc[(el2011['pct_votes'] > el2011['threshold']) & (el2011['votes'] > 20), :]\
    .groupby(['dist_code', 'section_code', 'table_code'])\
    .agg({'pct_votes': 'count'})

pct_votes
dist_code section_code table_code           
01        062          U                   3
03        078          B                   1
05        091          U                   1
          093          U                   1
08        012          U                   4
09        018          B                   1
11        007          B                   2
12        101          A                   1
13        060          U                   1
          203          A                   3
18        032          A                   1
          048          B                   1